In [3]:
import sys
import os

# เพิ่ม path ของโฟลเดอร์หลักเข้าไปใน sys.path
sys.path.append(os.path.abspath(".."))

# ตอนนี้สามารถ import ได้เลย
from main_graph import graph
from state_schema import AgentState


In [4]:
import pandas as pd
from tqdm import tqdm

In [ ]:
import sys
import os
import pandas as pd
from tqdm import tqdm
import asyncio
import nest_asyncio

nest_asyncio.apply()  # ⭐️ สำคัญสำหรับ Jupyter / IPython

sys.path.append(os.path.abspath(".."))

# โหลด DataFrame
df = pd.read_excel("question_2categorylevel.xlsx")
df.columns = df.columns.str.strip()

results = []

async def process_query(query, expected_lv1, expected_lv2):
    try:
        initial_state = AgentState(user_query=query)
        state_raw = await graph.ainvoke(initial_state)
        state_obj = AgentState(**state_raw)

        classification = getattr(state_obj, "classification_result", None)
        if classification is None:
            # ถ้าไม่มี classification_result
            classification = {}

        out_of_domain = getattr(classification, "out_of_domain", False)
        out_of_domain_reason = getattr(classification, "out_of_domain_reason", "")

        # ถ้าไม่มีเหตุผล ให้ใส่ default
        if out_of_domain and not out_of_domain_reason:
            out_of_domain_reason = "Out-of-domain without specific reason."

        if out_of_domain:
            predicted_lv1 = "OUT_OF_DOMAIN"
            predicted_lv2 = "OUT_OF_DOMAIN"
            correct_lv1 = expected_lv1.strip().upper() == "OUT_OF_DOMAIN"
            correct_lv2 = expected_lv2.strip().upper() == "OUT_OF_DOMAIN"

            # log กรณี Out-of-domain
            print(f"🔹 Out-of-domain detected: {query}")
            print(f"   Reason: {out_of_domain_reason}")

        else:
            cat_lv1 = getattr(classification, "category_level_1", []) or []
            if cat_lv1 and len(cat_lv1) > 0:
                predicted_lv1 = cat_lv1[0].category
            else:
                predicted_lv1 = ""

            cat_lv2 = getattr(classification, "category_level_2", []) or []
            if cat_lv2 and len(cat_lv2) > 0:
                predicted_lv2 = cat_lv2[0].subcategory
            else:
                predicted_lv2 = ""

            correct_lv1 = predicted_lv1 == expected_lv1
            correct_lv2 = predicted_lv2 == expected_lv2

        latency = state_obj.latency or {}
        total_latency = round(
            sum(v for k, v in latency.items() if isinstance(v, float)), 3
        )

        return {
            "query": query,
            "expected_lv1": expected_lv1,
            "predicted_lv1": predicted_lv1,
            "correct_lv1": correct_lv1,
            "expected_lv2": expected_lv2,
            "predicted_lv2": predicted_lv2,
            "correct_lv2": correct_lv2,
            "out_of_domain": out_of_domain,
            "out_of_domain_reason": out_of_domain_reason,
            "latency_out_of_domain": latency.get("out_of_domain"),
            "latency_clarification": latency.get("clarification"),
            "latency_classification": latency.get("classification"),
            "tokens_out_of_domain": latency.get("tokens_out_of_domain"),
            "tokens_clarification": latency.get("tokens_clarification"),
            "tokens_classification": latency.get("tokens_classification"),
            "total_latency": total_latency,
            "error": ""
        }
    except Exception as e:
        return {
            "query": query,
            "expected_lv1": expected_lv1,
            "predicted_lv1": "",
            "correct_lv1": False,
            "expected_lv2": expected_lv2,
            "predicted_lv2": "",
            "correct_lv2": False,
            "out_of_domain": None,
            "out_of_domain_reason": "",
            "latency_out_of_domain": None,
            "latency_clarification": None,
            "latency_classification": None,
            "tokens_out_of_domain": None,
            "tokens_clarification": None,
            "tokens_classification": None,
            "total_latency": None,
            "error": str(e)
        }

async def main():
    for _, row in tqdm(df.iterrows(), total=len(df)):
        query = row["question"]
        expected_lv1 = row["category_level1"]
        expected_lv2 = row["category_level2"]
        res = await process_query(query, expected_lv1, expected_lv2)
        results.append(res)

    df_results = pd.DataFrame(results)
    df_results.to_csv("evaluation_results.csv", index=False, encoding="utf-8-sig")
    print("✅ Evaluation completed and saved to evaluation_results.csv")

# ⭐️ ใน Jupyter Notebook ให้รันด้วย:
await main()


  0%|          | 0/20 [00:00<?, ?it/s]


KeyError: 'category_level1'

In [ ]:
results

[{'query': ' ยาชาถอนฟันจะอยู่ได้นานกี่ชั่วโมง?',
  'expected_lv1': 'หัตถการ',
  'predicted_lv1': 'อาการ/ภาวะแทรกซ้อน',
  'correct_lv1': False,
  'expected_lv2': 'การใช้ยาชา',
  'predicted_lv2': 'wound healing',
  'correct_lv2': False,
  'error': ''},
 {'query': 'หลังถอนฟัน อาการผิดปกติแบบไหนที่ควรกลับไปพบทันตแพทย์',
  'expected_lv1': 'อาการ/ภาวะแทรกซ้อน',
  'predicted_lv1': 'อาการ/ภาวะแทรกซ้อน',
  'correct_lv1': True,
  'expected_lv2': 'overall',
  'predicted_lv2': 'wound healing',
  'correct_lv2': False,
  'error': ''},
 {'query': 'ความเสี่ยงและผลกระทบที่เกิดจากการถอนฟัน',
  'expected_lv1': 'อาการ/ภาวะแทรกซ้อน',
  'predicted_lv1': 'อาการ/ภาวะแทรกซ้อน',
  'correct_lv1': True,
  'expected_lv2': 'overall',
  'predicted_lv2': 'wound healing',
  'correct_lv2': False,
  'error': ''},
 {'query': 'สัญญาณของการติดเชื้อหลังถอนฟัน',
  'expected_lv1': 'อาการ/ภาวะแทรกซ้อน',
  'predicted_lv1': 'อาการ/ภาวะแทรกซ้อน',
  'correct_lv1': True,
  'expected_lv2': 'overall',
  'predicted_lv2': 'wound healin

In [6]:
import pandas as pd

# สมมุติว่า results เป็น list ของ dictionary
results_df4 = pd.DataFrame(results)

# แล้วค่อยบันทึกเป็น Excel
results_df4.to_excel("results_outOfDomain_openAI.xlsx", index=False)



In [20]:
import pandas as pd

# สมมุติว่า results เป็น list ของ dictionary
results_df = pd.DataFrame(results)

# แล้วค่อยบันทึกเป็น Excel
results_df.to_excel("results.xlsx", index=False)



In [12]:
print(df.columns.tolist())


['category_level1', 'category_level2', 'question']


In [18]:
from tqdm import tqdm
import pandas as pd

df = pd.read_excel("question_clari_test.xlsx")
df.columns = df.columns.str.strip()
df["clarification_needed_expected"] = True
results_clari = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    query = row["question"]
    expected_clarification = row["clarification_needed_expected"]

    try:
        initial_state = AgentState(user_query=query)
        state = graph.invoke(initial_state)
        state_obj = AgentState(**state)

        # ✅ ดึงผลลัพธ์จาก state
        predicted_clarification = getattr(state_obj.classification_result, "clarification_needed", None)
        clarification_reason = getattr(state_obj.classification_result, "clarification_reason", "")
        out_of_domain = getattr(state_obj.classification_result, "out_of_domain", None)
        out_of_domain_reason = getattr(state_obj.classification_result, "out_of_domain_reason", "")

        correct_clarification = predicted_clarification == expected_clarification

        latency = state_obj.latency or {}
        total_latency = round(sum(v for k, v in latency.items() if isinstance(v, float)), 3)

        # ✅ เพิ่ม note เฉพาะกรณี out_of_domain=True
        note = ""
        if out_of_domain is True:
            note = f"out_of_domain = True, reason: {out_of_domain_reason}"

        results_clari.append({
            "query": query,
            "expected_clarification": expected_clarification,
            "predicted_clarification": predicted_clarification,
            "correct_clarification": correct_clarification,
            "clarification_reason": clarification_reason,
            "latency_clarification": latency.get("clarification"),
            "tokens_clarification": latency.get("tokens_clarification"),
            "total_latency": total_latency,
            "note": note,
            "error": ""
        })

    except Exception as e:
        results_clari.append({
            "query": query,
            "expected_clarification": expected_clarification,
            "predicted_clarification": "",
            "correct_clarification": False,
            "clarification_reason": "",
            "latency_clarification": None,
            "tokens_clarification": None,
            "total_latency": None,
            "note": "",
            "error": str(e)
        })


  0%|          | 0/15 [00:00<?, ?it/s]

🚦 Step 1: Out-of-Domain Check


  7%|▋         | 1/15 [00:02<00:29,  2.08s/it]

✅ Out-of-Domain Result: out_of_domain=True reason='ไม่พบคำที่เกี่ยวข้องกับทันตกรรมหรือบริบทหลังการรักษา'
🚦 Step 1: Out-of-Domain Check
✅ Out-of-Domain Result: out_of_domain=False reason='มีคำว่า อย่างอย่างเป็นย์'
🔍 Step 2: Clarity Check


 13%|█▎        | 2/15 [00:23<02:55, 13.47s/it]

...

✅ Out-of-Domain Result: out_of_domain=True reason='ไม่เกี่ยวข้องกับทันตกรรม'
🚦 Step 1: Out-of-Domain Check


 93%|█████████▎| 14/15 [02:50<00:11, 11.74s/it]

✅ Out-of-Domain Result: out_of_domain=True reason='ไม่พบคำที่เกี่ยวข้องกับทันตกรรมหรือบริบทหลังการรักษา'
🚦 Step 1: Out-of-Domain Check


100%|██████████| 15/15 [02:57<00:00, 11.85s/it]

✅ Out-of-Domain Result: out_of_domain=True reason='ไม่มีบริบทเกี่ยวกับการดูแลหลังหัตถการ'


In [15]:
results_clari

[{'query': 'ต้องพักกี่วันถึงจะหายดี',
  'expected_clarification': True,
  'predicted_clarification': None,
  'correct_clarification': False,
  'clarification_reason': '',
  'latency_clarification': None,
  'tokens_clarification': None,
  'total_latency': 24.04,
  'note': 'out_of_domain = True, reason: ',
  'error': ''},
 {'query': 'ยังมีเลือดซึมอยู่นิดหน่อย',
  'expected_clarification': True,
  'predicted_clarification': True,
  'correct_clarification': True,
  'clarification_reason': '',
  'latency_clarification': 9.536,
  'tokens_clarification': 26.0,
  'total_latency': 74.97,
  'note': '',
  'error': ''},
 {'query': 'กินเผ็ดได้ไหมคะ',
  'expected_clarification': True,
  'predicted_clarification': None,
  'correct_clarification': False,
  'clarification_reason': '',
  'latency_clarification': None,
  'tokens_clarification': None,
  'total_latency': 17.241,
  'note': 'out_of_domain = True, reason: ',
  'error': ''},
 {'query': 'ทำไมยังบวมอยู่',
  'expected_clarification': True,
  'pre

In [19]:
import pandas as pd

# สมมุติว่า results เป็น list ของ dictionary
results_clari = pd.DataFrame(results_clari)

# แล้วค่อยบันทึกเป็น Excel
results_clari.to_excel("results_clari3.xlsx", index=False)

In [8]:
import sys
import os
import pandas as pd
from tqdm import tqdm
import asyncio

# ถ้า AgentState และ graph มาจากไฟล์อื่น ให้ import ให้ถูก
# from your_module import AgentState, graph

sys.path.append(os.path.abspath(".."))

# โหลด DataFrame
df = pd.read_excel("question_outofdomain_test.xlsx")
df.columns = df.columns.str.strip()

results = []

async def process_query(query, expected_lv1, expected_lv2):
    try:
        initial_state = AgentState(user_query=query)
        state_raw = await graph.ainvoke(initial_state)
        state_obj = AgentState(**state_raw)

        classification = getattr(state_obj, "classification_result", None)
        if classification is None:
            classification = {}

        out_of_domain = getattr(classification, "out_of_domain", False)
        out_of_domain_reason = getattr(classification, "out_of_domain_reason", "")

        if out_of_domain and not out_of_domain_reason:
            out_of_domain_reason = "Out-of-domain without specific reason."

        if out_of_domain:
            predicted_lv1 = "OUT_OF_DOMAIN"
            predicted_lv2 = "OUT_OF_DOMAIN"
            correct_lv1 = expected_lv1.strip().upper() == "OUT_OF_DOMAIN"
            correct_lv2 = expected_lv2.strip().upper() == "OUT_OF_DOMAIN"

            print(f"🔹 Out-of-domain detected: {query}")
            print(f"   Reason: {out_of_domain_reason}")

        else:
            cat_lv1 = getattr(classification, "category_level_1", []) or []
            if cat_lv1 and len(cat_lv1) > 0:
                predicted_lv1 = cat_lv1[0].category
            else:
                predicted_lv1 = ""

            cat_lv2 = getattr(classification, "category_level_2", []) or []
            if cat_lv2 and len(cat_lv2) > 0:
                predicted_lv2 = cat_lv2[0].subcategory
            else:
                predicted_lv2 = ""

            correct_lv1 = predicted_lv1 == expected_lv1
            correct_lv2 = predicted_lv2 == expected_lv2

        latency = state_obj.latency or {}
        total_latency = round(
            sum(v for k, v in latency.items() if isinstance(v, float)), 3
        )

        return {
            "query": query,
            "expected_lv1": expected_lv1,
            "predicted_lv1": predicted_lv1,
            "correct_lv1": correct_lv1,
            "expected_lv2": expected_lv2,
            "predicted_lv2": predicted_lv2,
            "correct_lv2": correct_lv2,
            "out_of_domain": out_of_domain,
            "out_of_domain_reason": out_of_domain_reason,
            "latency_out_of_domain": latency.get("out_of_domain"),
            "latency_clarification": latency.get("clarification"),
            "latency_classification": latency.get("classification"),
            "tokens_out_of_domain": latency.get("tokens_out_of_domain"),
            "tokens_clarification": latency.get("tokens_clarification"),
            "tokens_classification": latency.get("tokens_classification"),
            "total_latency": total_latency,
            "error": ""
        }
    except Exception as e:
        return {
            "query": query,
            "expected_lv1": expected_lv1,
            "predicted_lv1": "",
            "correct_lv1": False,
            "expected_lv2": expected_lv2,
            "predicted_lv2": "",
            "correct_lv2": False,
            "out_of_domain": None,
            "out_of_domain_reason": "",
            "latency_out_of_domain": None,
            "latency_clarification": None,
            "latency_classification": None,
            "tokens_out_of_domain": None,
            "tokens_clarification": None,
            "tokens_classification": None,
            "total_latency": None,
            "error": str(e)
        }

async def main():
    for _, row in tqdm(df.iterrows(), total=len(df)):
        query = row["question"]
        expected_lv1 = row["category_level1"]
        expected_lv2 = row["category_level2"]
        res = await process_query(query, expected_lv1, expected_lv2)
        results.append(res)

    df_results = pd.DataFrame(results)
    df_results.to_csv("evaluation_results.csv", index=False, encoding="utf-8-sig")
    print("✅ Evaluation completed and saved to evaluation_results.csv")

if __name__ == "__main__":
    asyncio.run(main())


  0%|          | 0/20 [00:00<?, ?it/s]


KeyError: 'category_level1'

In [11]:
import sys
import os
import pandas as pd
from tqdm import tqdm
import asyncio

# ✨ ถ้า AgentState และ graph อยู่ในโมดูลอื่น ให้แก้ตรงนี้
# from your_module import AgentState, graph

sys.path.append(os.path.abspath(".."))

# โหลด DataFrame
df = pd.read_excel("question_outofdomain_test.xlsx")
df.columns = df.columns.str.strip()

results = []

async def process_query(query, expected_lv1, expected_lv2):
    try:
        initial_state = AgentState(user_query=query)
        state_raw = await graph.ainvoke(initial_state)
        state_obj = AgentState(**state_raw)

        classification = getattr(state_obj, "classification_result", None)
        if classification is None:
            classification = {}

        out_of_domain = getattr(classification, "out_of_domain", False)
        out_of_domain_reason = getattr(classification, "out_of_domain_reason", "")

        if out_of_domain and not out_of_domain_reason:
            out_of_domain_reason = "Out-of-domain without specific reason."

        if out_of_domain:
            predicted_lv1 = "OUT_OF_DOMAIN"
            predicted_lv2 = "OUT_OF_DOMAIN"
            correct_lv1 = expected_lv1.strip().upper() == "OUT_OF_DOMAIN"
            correct_lv2 = expected_lv2.strip().upper() == "OUT_OF_DOMAIN"

            print(f"🔹 Out-of-domain detected: {query}")
            print(f"   Reason: {out_of_domain_reason}")

        else:
            cat_lv1 = getattr(classification, "category_level_1", []) or []
            if cat_lv1 and len(cat_lv1) > 0:
                predicted_lv1 = cat_lv1[0].category
            else:
                predicted_lv1 = ""

            cat_lv2 = getattr(classification, "category_level_2", []) or []
            if cat_lv2 and len(cat_lv2) > 0:
                predicted_lv2 = cat_lv2[0].subcategory
            else:
                predicted_lv2 = ""

            correct_lv1 = predicted_lv1 == expected_lv1
            correct_lv2 = predicted_lv2 == expected_lv2

        latency = state_obj.latency or {}
        total_latency = round(
            sum(v for k, v in latency.items() if isinstance(v, float)), 3
        )

        return {
            "query": query,
            "expected_lv1": expected_lv1,
            "predicted_lv1": predicted_lv1,
            "correct_lv1": correct_lv1,
            "expected_lv2": expected_lv2,
            "predicted_lv2": predicted_lv2,
            "correct_lv2": correct_lv2,
            "out_of_domain": out_of_domain,
            "out_of_domain_reason": out_of_domain_reason,
            "latency_out_of_domain": latency.get("out_of_domain"),
            "latency_clarification": latency.get("clarification"),
            "latency_classification": latency.get("classification"),
            "tokens_out_of_domain": latency.get("tokens_out_of_domain"),
            "tokens_clarification": latency.get("tokens_clarification"),
            "tokens_classification": latency.get("tokens_classification"),
            "total_latency": total_latency,
            "error": ""
        }
    except Exception as e:
        return {
            "query": query,
            "expected_lv1": expected_lv1,
            "predicted_lv1": "",
            "correct_lv1": False,
            "expected_lv2": expected_lv2,
            "predicted_lv2": "",
            "correct_lv2": False,
            "out_of_domain": None,
            "out_of_domain_reason": "",
            "latency_out_of_domain": None,
            "latency_clarification": None,
            "latency_classification": None,
            "tokens_out_of_domain": None,
            "tokens_clarification": None,
            "tokens_classification": None,
            "total_latency": None,
            "error": str(e)
        }

async def main():
    print("Columns in DataFrame:", df.columns.tolist())

    for _, row in tqdm(df.iterrows(), total=len(df)):
        query = row["question"]

        # ตรวจว่ามีเหตุผล out-of-domain หรือไม่
        expected_out_of_domain_reason = str(row["เหตุผลที่อยู่นอกขอบเขต (Out of Domain)"]).strip()
        expected_is_out_of_domain = bool(expected_out_of_domain_reason)

        # ถ้าเป็น out-of-domain ให้ expected lv1/lv2 เป็น OUT_OF_DOMAIN
        expected_lv1 = "OUT_OF_DOMAIN" if expected_is_out_of_domain else ""
        expected_lv2 = "OUT_OF_DOMAIN" if expected_is_out_of_domain else ""

        res = await process_query(query, expected_lv1, expected_lv2)
        results.append(res)

    df_results = pd.DataFrame(results)
    df_results.to_csv("evaluation_results.csv", index=False, encoding="utf-8-sig")
    print("✅ Evaluation completed and saved to evaluation_results.csv")

if __name__ == "__main__":
    asyncio.run(main())


Columns in DataFrame: ['question', 'เหตุผลที่อยู่นอกขอบเขต (Out of Domain)']


  5%|▌         | 1/20 [00:02<00:46,  2.44s/it]

🔹 Out-of-domain detected: วันนี้ฝนจะตกไหมคะ
   Reason: คำถามไม่เกี่ยวข้องกับทันตกรรมหรือการดูแลหลังหัตถการ
🔍 Step 1: Clarity Check


 10%|█         | 2/20 [00:06<01:03,  3.50s/it]

✅ Clarity Result: clarification_needed=True reason='ไม่มีบริบททางทันตกรรม'


...

✅ Clarity Result: clarification_needed=True reason='ไม่มีเจตนาถาม'


 95%|█████████▌| 19/20 [01:08<00:04,  4.55s/it]

🔹 Out-of-domain detected: Dream กับ Goal ต่างกันยังไง
   Reason: คำถามไม่เกี่ยวกับหลังหัตถการทางทันตกรรม และไม่มีบริบทเกี่ยวกับการดูแลหลังหัตถการ


100%|██████████| 20/20 [01:10<00:00,  3.52s/it]

🔹 Out-of-domain detected: อยากทำหน้า V-shape ควรฉีดอะไรดี
   Reason: คำถามเกี่ยวกับการฉีดเพื่อทำหน้า V-shape ไม่เกี่ยวกับทันตกรรมหลังหัตถการ
✅ Evaluation completed and saved to evaluation_results.csv
